# LlamaAgent: Getting Started

Welcome to LlamaAgent - a comprehensive AI agent framework with OpenAI Agents SDK integration!

This notebook will guide you through:
1. Basic agent creation and execution
2. Using tools with agents
3. Multi-agent orchestration
4. OpenAI integration
5. Advanced features (SPRE, budget tracking, etc.)

**Author:** Nik Jois <nikjois@llamasearch.ai>

## 1. Setup and Installation

First, let's ensure all dependencies are installed:

In [ ]:
# Install LlamaAgent (if not already installed)
!pip install -e ..

# Import required modules
import os

# LlamaAgent imports
from llamaagent import AgentConfig, AgentRole, ReactAgent
from llamaagent.llm import create_provider
from llamaagent.tools import ToolRegistry, get_all_tools

print("✓ LlamaAgent imported successfully")

## 2. Creating Your First Agent

Let's create a simple agent that can perform calculations:

In [ ]:
# Create agent configuration
config = AgentConfig(
    name="CalculatorAgent",
    role=AgentRole.SPECIALIST,
    description="An agent that performs mathematical calculations",
    max_iterations=5,
    temperature=0.1
)

# Create the agent with mock provider (no API key needed)
agent = ReactAgent(
    config=config,
    llm_provider=create_provider("mock")
)

print(f"✓ Created agent: {agent.name}")
print(f"  Role: {config.role.value}")
print(f"  Description: {config.description}")

In [ ]:
# Execute a simple task
async def run_calculation():
    task = "Calculate the sum of squares from 1 to 10"
    response = await agent.execute(task)
    
    print(f"Task: {task}")
    print(f"Success: {response.success}")
    print(f"Response: {response.content}")
    print(f"Execution time: {response.execution_time:.2f}s")
    
await run_calculation()

## 3. Adding Tools to Agents

Tools give agents the ability to perform specific actions:

In [ ]:
# Create a tool registry and add tools
tools = ToolRegistry()

# Get all available tools
available_tools = get_all_tools()
for tool in available_tools:
    tools.register(tool)
    print(f"✓ Registered tool: {tool.name}")

# Create an agent with tools
tool_agent_config = AgentConfig(
    name="ToolAgent",
    role=AgentRole.EXECUTOR,
    description="An agent that can use various tools",
    tools=["calculator", "python_repl"]
)

tool_agent = ReactAgent(
    config=tool_agent_config,
    tools=tools,
    llm_provider=create_provider("mock")
)

print(f"\n✓ Created agent with {len(tools.list_tools())} tools")

In [ ]:
# Execute a task that requires tools
async def run_tool_task():
    task = """Write a Python function to calculate factorial and 
    then use it to find the factorial of 10"""
    
    response = await tool_agent.execute(task)
    
    print(f"Task: {task}")
    print(f"\nSuccess: {response.success}")
    print(f"\nResponse: {response.content}")
    
    # Show trace of tool usage
    if response.trace:
        tool_calls = [t for t in response.trace if t.get('type') == 'tool_execution_start']
        print(f"\nTools used: {len(tool_calls)}")
        for call in tool_calls:
            print(f"  - {call.get('data', {}).get('tool_name', 'Unknown')}")
    
await run_tool_task()

## 4. Multi-Agent Orchestration

LlamaAgent supports coordinating multiple agents to solve complex tasks:

In [ ]:
from llamaagent.orchestrator import AgentOrchestrator, OrchestrationStrategy, WorkflowDefinition, WorkflowStep

# Create orchestrator
orchestrator = AgentOrchestrator()

# Create specialized agents
agents = {
    "researcher": ReactAgent(
        config=AgentConfig(
            name="ResearchAgent",
            role=AgentRole.RESEARCHER,
            description="Gathers information"
        ),
        llm_provider=create_provider("mock")
    ),
    "analyst": ReactAgent(
        config=AgentConfig(
            name="AnalystAgent",
            role=AgentRole.ANALYZER,
            description="Analyzes data"
        ),
        llm_provider=create_provider("mock")
    ),
    "writer": ReactAgent(
        config=AgentConfig(
            name="WriterAgent",
            role=AgentRole.SPECIALIST,
            description="Creates reports"
        ),
        llm_provider=create_provider("mock")
    )
}

# Register agents
for agent in agents.values():
    orchestrator.register_agent(agent)
    print(f"✓ Registered {agent.name}")

In [ ]:
# Define a workflow
workflow = WorkflowDefinition(
    workflow_id="report_workflow",
    name="Research Report Workflow",
    description="Create a research report on AI trends",
    strategy=OrchestrationStrategy.SEQUENTIAL,
    steps=[
        WorkflowStep(
            step_id="research",
            agent_name="ResearchAgent",
            task="Research the latest trends in AI and machine learning"
        ),
        WorkflowStep(
            step_id="analyze",
            agent_name="AnalystAgent",
            task="Analyze the research findings and identify key patterns",
            dependencies=["research"]
        ),
        WorkflowStep(
            step_id="write",
            agent_name="WriterAgent",
            task="Write a concise report summarizing the findings",
            dependencies=["research", "analyze"]
        )
    ]
)

# Register and execute workflow
orchestrator.register_workflow(workflow)

async def run_workflow():
    print(f"Executing workflow: {workflow.name}\n")
    
    result = await orchestrator.execute_workflow(workflow.workflow_id)
    
    print("\nWorkflow completed!")
    print(f"Success: {result.success}")
    print(f"Total time: {result.execution_time:.2f}s")
    
    # Show results from each step
    for step_id, step_result in result.results.items():
        print(f"\n{step_id}: {step_result.status.value}")
        if step_result.result and step_result.result.data:
            print(f"  Output preview: {str(step_result.result.data)[:100]}...")

await run_workflow()

## 5. OpenAI Integration

Now let's explore the OpenAI Agents SDK integration:

In [ ]:
# Check if OpenAI API key is available
if os.getenv("OPENAI_API_KEY"):
    print("✓ OpenAI API key found")
    use_openai = True
else:
    print("✗ No OpenAI API key found - using mock provider")
    print("  Set OPENAI_API_KEY environment variable to use OpenAI")
    use_openai = False

In [ ]:
from llamaagent.integration.openai_agents import create_openai_integration

if use_openai:
    # Create OpenAI integration
    openai_integration = create_openai_integration(
        model_name="gpt-4o-mini",
        budget_limit=1.0  # $1 budget for demo
    )
    
    # Create an agent with OpenAI
    openai_agent = ReactAgent(
        config=AgentConfig(
            name="OpenAIAgent",
            role=AgentRole.GENERALIST,
            description="Powered by OpenAI GPT-4"
        ),
        llm_provider=create_provider("openai", model_name="gpt-4o-mini")
    )
    
    # Register with integration
    adapter = openai_integration.register_agent(openai_agent)
    
    print("✓ OpenAI integration configured")
    print("  Model: gpt-4o-mini")
    print("  Budget: $1.00")
else:
    print("Skipping OpenAI integration (no API key)")

In [ ]:
# Execute a task with OpenAI (if available)
if use_openai and 'openai_integration' in locals():
    from llamaagent.types import TaskInput
    
    async def run_openai_task():
        task = TaskInput(
            id="openai_demo_001",
            task="Explain quantum computing in simple terms",
            agent_name="OpenAIAgent"
        )
        
        result = await openai_integration.run_task("OpenAIAgent", task)
        
        print(f"Task: {task.task}")
        print(f"\nSuccess: {result.success}")
        if result.result and result.result.data:
            print(f"\nResponse: {result.result.data.get('response', 'No response')}")
        
        # Show budget usage
        budget = openai_integration.get_budget_status()
        print(f"\nBudget used: ${budget['current_cost']:.4f}")
        print(f"Remaining: ${budget['remaining_budget']:.4f}")
    
    await run_openai_task()
else:
    print("OpenAI integration not available - using mock response")
    print("\nMock Response: Quantum computing uses quantum bits (qubits) that can be ")
    print("in multiple states simultaneously, unlike classical bits that are only 0 or 1.")

## 6. Advanced Features: SPRE (Strategic Planning & Resourceful Execution)

SPRE is LlamaAgent's advanced planning methodology:

In [ ]:
# Create a SPRE-enabled agent
spre_config = AgentConfig(
    name="SPREAgent",
    role=AgentRole.PLANNER,
    description="Agent with Strategic Planning & Resourceful Execution",
    spree_enabled=True,  # Enable SPRE
    tools=["calculator", "python_repl"]
)

spre_agent = ReactAgent(
    config=spre_config,
    tools=tools,
    llm_provider=create_provider("mock")
)

print("✓ Created SPRE-enabled agent")
print(f"  SPRE enabled: {spre_config.spree_enabled}")

In [ ]:
# Execute a complex task with SPRE
async def run_spre_task():
    complex_task = """Calculate the compound interest on $5000 at 8% for 5 years,
    then determine how much you need to invest at the same rate to have $10000 after 5 years."""
    
    print(f"Task: {complex_task}\n")
    
    response = await spre_agent.execute(complex_task)
    
    print(f"Success: {response.success}")
    print(f"\nResponse: {response.content}")
    
    # Show planning events
    if response.trace:
        planning_events = [e for e in response.trace if e.get('type') == 'plan_generated']
        if planning_events:
            print(f"\nSPRE Planning detected: {len(planning_events)} plan(s) generated")

await run_spre_task()

## 7. Monitoring and Debugging

LlamaAgent provides comprehensive monitoring capabilities:

In [ ]:
# Create an agent with detailed tracing
debug_config = AgentConfig(
    name="DebugAgent",
    role=AgentRole.SPECIALIST,
    description="Agent with detailed debugging",
    debug=True,
    verbose=True
)

debug_agent = ReactAgent(
    config=debug_config,
    llm_provider=create_provider("mock")
)

# Execute and analyze trace
async def debug_execution():
    response = await debug_agent.execute("What is 2 + 2?")
    
    print(f"Response: {response.content}")
    
    # Get execution trace
    trace = debug_agent.get_trace()
    if trace:
        print("\nExecution Trace:")
        print(f"  Agent: {trace.agent_name}")
        print(f"  Task: {trace.task}")
        print(f"  Execution time: {trace.execution_time:.3f}s")
        print(f"  Success: {trace.success}")
        print(f"  Steps: {len(trace.steps)}")
        
        for i, step in enumerate(trace.steps):
            print(f"\n  Step {i+1}: {step['step_type']}")
            print(f"    {step['description']}")

await debug_execution()

## 8. Best Practices and Tips

Here are some best practices for using LlamaAgent effectively:

In [ ]:
# Best Practice 1: Use appropriate agent roles
print("Available Agent Roles:")
for role in AgentRole:
    print(f"  - {role.value}: {role.name}")

print("\nTip: Choose roles that match the agent's primary function")

In [ ]:
# Best Practice 2: Configure agents appropriately
optimal_config = AgentConfig(
    name="OptimalAgent",
    role=AgentRole.SPECIALIST,
    description="Well-configured agent",
    max_iterations=10,      # Prevent infinite loops
    temperature=0.1,        # Low temperature for consistency
    timeout=60.0,          # Reasonable timeout
    retry_attempts=3,       # Handle transient failures
    memory_enabled=True,    # Enable conversation memory
    streaming=False,        # Disable for batch processing
)

print("Optimal Configuration:")
for key, value in optimal_config.__dict__.items():
    if not key.startswith('_'):
        print(f"  {key}: {value}")

In [ ]:
# Best Practice 3: Handle errors gracefully
async def error_handling_example():
    agent = ReactAgent(
        config=AgentConfig(name="ErrorAgent"),
        llm_provider=create_provider("mock")
    )
    
    # Test with an edge case
    try:
        response = await agent.execute("")
        if response.success:
            print("Empty task handled successfully")
        else:
            print(f"Task failed gracefully: {response.error}")
    except Exception as e:
        print(f"Exception caught: {type(e).__name__}: {e}")

await error_handling_example()

## 9. Conclusion

You've now learned the basics of LlamaAgent! Here's what we covered:

1. ✅ Creating and configuring agents
2. ✅ Using tools with agents
3. ✅ Multi-agent orchestration
4. ✅ OpenAI integration
5. ✅ SPRE methodology
6. ✅ Monitoring and debugging
7. ✅ Best practices

### Next Steps:

- Explore more examples in the `/examples` directory
- Read the full documentation
- Try building your own multi-agent system
- Contribute to the project on GitHub

Happy agent building! 🦙

In [ ]:
# Clean up
print("Notebook completed successfully!")
print("\nFor questions or support:")
print("  Email: nikjois@llamasearch.ai")
print("  GitHub: https://github.com/nikjois/llamaagent")